In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, Output
from amplify import Solver, BinarySymbolGenerator, BinaryQuadraticModel
from amplify.client import FixstarsClient
from amplify.constraint import penalty, one_hot

client = FixstarsClient()

%matplotlib inline

# グラフ彩色問題

グラフ彩色問題とは隣接する領域同士が同じ色にならないように全領域を塗り分ける問題です。

この問題の応用例の一部として下記が挙げられます。

* 会議室の割り当てや担当業務の割り当てなどのスケジューリング問題
* 電波の周波数帯などの共有リソースの割り当て問題

次のデモは日本の都道府県を異なる色で塗り分ける問題を QUBO 定式化を用いて Amplify Annealing Engine で実行します。  
「Run」ボタンをクリックすると出力された解が表示されます。

In [ ]:
import japanmap as jm


def coloring_initialize():
    plt.rcParams["figure.figsize"] = 7.6, 7.6
    plt.imshow(jm.picture())
    plt.show()


def coloring_solve(*args):
    colors = ["red", "green", "blue", "yellow"]
    num_colors = len(colors)
    num_region = len(jm.pref_names) - 1  # 都道府県数を取得

    gen = BinarySymbolGenerator()
    q = gen.array(num_region, num_colors)

    # 各領域に対する制約
    reg_constraints = [one_hot(q[i]) for i in range(num_region)]

    # 隣接する領域間の制約
    adj_constraints = [
        # 都道府県コードと配列インデックスは1ずれてるので注意
        penalty(q[i, c] * q[j - 1, c])
        for i in range(num_region)
        for j in jm.adjacent(i + 1)  # j: 隣接している都道府県コード
        if i + 1 < j
        for c in range(num_colors)
    ]

    constraints = sum(reg_constraints) + sum(adj_constraints)

    solver = Solver(client)
    solver.client.parameters.timeout = 1000

    model = BinaryQuadraticModel(constraints)

    result = solver.solve(model)
    i = 0
    while len(result) == 0:
        if i > 5:
            raise RuntimeError()
        result = solver.solve(model)
        i += 1

    values = result[0].values
    q_values = q.decode(values)
    color_indices = np.where(np.array(q_values) == 1)[1]
    color_map = {
        jm.pref_names[i + 1]: colors[color_indices[i]]
        for i in range(len(color_indices))
    }

    plt.rcParams["figure.figsize"] = 7.6, 7.6
    plt.imshow(jm.picture(color_map))
    plt.show()

In [ ]:
coloring_run_btn = Button(
    description="Run", button_style="", tooltip="Run", icon="check"
)
coloring_problem_out = Output()
coloring_result_out = Output()


def show_coloring_problem():
    with coloring_problem_out:
        coloring_initialize()


def show_coloring_result(btn):
    with coloring_result_out:
        coloring_result_out.clear_output()
        coloring_solve()


coloring_run_btn.on_click(show_coloring_result)
display(HBox([coloring_run_btn]), HBox([coloring_problem_out, coloring_result_out]))
show_coloring_problem()